In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.lines import Line2D

from PyCo.ContactMechanics import HardWall
from PyCo.SolidMechanics import PeriodicFFTElasticHalfSpace, FreeFFTElasticHalfSpace
from PyCo.Topography import Topography
from PyCo.System import make_system

# Generate Cone

nx, ny = 128,128

sx = 0.005 # mm
sy = 0.005 # mm

x = np.arange(0, nx).reshape(-1,1 ) * sx/nx - sx / 2
y = np.arange(0, ny).reshape(1,-1 ) * sy/ny - sy / 2
X = x * np.ones_like(y)
Y = y * np.ones_like(x)

topography= Topography(- np.sqrt(x**2 + y**2 ) *0.1, physical_sizes=(sx, sy) )
Max_Height=(-1)*np.min(topography.heights())

fig, ax = plt.subplots()
plt.colorbar(ax.pcolormesh(X, Y , topography.heights()), label = "heights")
ax.set_aspect(1)
ax.set_xlabel("x (mm)")
ax.set_ylabel("y (mm)")

fig, ax = plt.subplots()
ax.plot(x, topography.heights()[:, ny//2])
ax.set_title("the max height is {0:.6f}mm".format(Max_Height))
ax.set_aspect(1)
ax.set_xlabel("x (mm)")
ax.set_ylabel("heights (mm)")

# Establish Foundamental Parameteres

# Defining Young's Module/ MPa
Es = 500
# Establishing PeriodicFFTElasticHalfSpace
substrate = FreeFFTElasticHalfSpace(nb_grid_pts=(nx,ny), young=Es, physical_sizes=(sx, sy))
interaction=HardWall()
# Establishing Interaction
system = make_system(substrate, interaction, topography)

# Based on PyCo Code

offset=[]
External_load=[]
Max_Pressure=[]
Contact_Area=[]

for Times in range(5):
    unit_depth=Max_Height/10
    offset.append(unit_depth*Times)
    sol=system.minimize_proxy(offset=offset[Times])
    assert sol.success
    Contact_Area.append(system.compute_contact_area())

    External_load.append(system.compute_normal_force())

    Deformation=(-1)*substrate.evaluate_disp(system.force) # make deformation positive
    
    Max_Pressure.append(np.max(system.force / system.area_per_pt))

    fig,(ax3D,ax3Dx,ax3Dy)=plt.subplots(1,3,figsize=(14,5))
    plt.subplots_adjust(wspace=0.5,hspace=0)
    pcm=ax3D.pcolormesh(X,Y, system.force / system.area_per_pt)
    plt.colorbar(pcm, ax = ax3D, label= "pressure (MPa)")
    ax3D.set_title("Moving Dis={0:.6f}mm".format(offset[Times]))
    ax3D.set_aspect(1)

    for Cro_Section in [ny//4,ny//2,ny*3//4]:
        P,=ax3Dx.plot(X[:,Cro_Section],system.force[:,Cro_Section] / system.area_per_pt)
        ax3Dx.set_ylabel('Pressure (MPa)')
        D,=ax3Dy.plot(X[:,Cro_Section],Deformation[:,Cro_Section],label='Deformation (mm)')
        ax3Dy.set_ylabel('Deformation (mm)')
        ax3D.axhline(Y[0,Cro_Section],color=P.get_color())
        ax3D.axhline(Y[0,Cro_Section],color=D.get_color())

# Based on Formula

angle=np.arctan(10)

F_offset=[]
F_External_load=[]
F_Max_Pressure=[]
F_Contact_Area=[]

for Times in range(5):
    unit_depth=Max_Height/10
    F_offset.append(unit_depth*Times)
    sol=system.minimize_proxy(offset=F_offset[Times], pentol=1e-5, prestol = 1e-5)
    assert sol.success
    
    F_Contact_Area.append(system.compute_contact_area())
    Total_Force=1/2 * Es * F_Contact_Area[Times] * np.tan(angle)**(-1)# useing formaular 5.14
    F_External_load.append(Total_Force)

    deformation=(-1)*substrate.evaluate_disp(system.force)
    
    F_Max_Pressure.append(np.max(system.force / system.area_per_pt))

    fig,(ax3D,ax3Dx,ax3Dy)=plt.subplots(1,3,figsize=(14,5))
    plt.subplots_adjust(wspace=0.5,hspace=0)
    pcm=ax3D.pcolormesh(X,Y, system.force / system.area_per_pt)
    plt.colorbar(pcm, ax = ax3D, label= "pressure (MPa)")
    ax3D.set_title("Moving Dis={0:.6f}mm".format(F_offset[Times]))
    ax3D.set_aspect(1)

    for Cro_Section in [ny//4,ny//2,ny*3//4]:
        P,=ax3Dx.plot(X[:,Cro_Section],system.force[:,Cro_Section] / system.area_per_pt)
        ax3Dx.set_ylabel('Pressure (MPa)')
        D,=ax3Dy.plot(X[:,Cro_Section],deformation[:,Cro_Section])
        ax3Dy.set_ylabel('Deformation (mm)')
        ax3D.axhline(Y[0,Cro_Section],color=P.get_color())
        ax3D.axhline(Y[0,Cro_Section],color=D.get_color())

# Plot of Offset & External_load

fig,(ax1,ad1)=plt.subplots(1,2,figsize=(14,5))

Difference_External_load=abs(np.array(External_load)-np.array(F_External_load))
ax1.plot(offset,External_load,color="green",marker="o",linestyle="dashed",linewidth=2,label="PyCo")
ax1.plot(F_offset,F_External_load,color="red",marker="s",linewidth=2,label="formula")
ax1.set_xlabel("offset(mm)")
ax1.set_ylabel("External_load(N)")
ax1.set_title("Offset & External_load between PyCo and Formulars")
ax1.legend()

ad1.plot(offset,Difference_External_load,color="green",marker="o",linewidth=2)
ad1.set_xlabel("offset(mm)")
ad1.set_ylabel("Difference in External_load(N)")
ad1.set_title("Difference External load between PyCo and Formulars")

# Plot of Offset & Max_Pressure

fig,(ax2,ad2)=plt.subplots(1,2,figsize=(14,5))
Difference_Max_Pressure=abs(np.array(Max_Pressure)-np.array(F_Max_Pressure))
ax2.plot(offset,Max_Pressure,color="green",marker="o",linestyle="dashed",linewidth=2,label="PyCo")
ax2.plot(F_offset,F_Max_Pressure,color="red",marker="s",linewidth=2,label="formula")
ax2.set_xlabel("offset(mm)")
ax2.set_ylabel("Max_Pressure(MPa)")
ax2.set_title("Offset & Max_Pressure between PyCo and Formulars")
ax2.legend()

ad2.plot(offset,Difference_Max_Pressure,color="green",marker="o",linewidth=2)
ad2.set_xlabel("offset(mm)")
ad2.set_ylabel("Difference in Max_Pressure(MPa)")
ad2.set_title("Difference Max Pressure between PyCo and Formulars")

# Plot of Offset & Contact_Area

fig,(ax3,ad3)=plt.subplots(1,2,figsize=(14,5))
Difference_Contact_Area=abs(np.array(Contact_Area)-np.array(F_Contact_Area))
ax3.plot(offset,Contact_Area,color="green",marker="o",linestyle="dashed",linewidth=2,label="PyCo")
ax3.plot(F_offset,F_Contact_Area,color="red",marker="s",linewidth=2,label="formula")
ax3.set_xlabel("offset(mm)")
ax3.set_ylabel("Contact_Area(mm2)")
ax3.set_title("Offset & Contact_Area between PyCo and Formulars")
ax3.legend()

ad3.plot(offset,Difference_Max_Pressure,color="green",marker="o",linewidth=2)
ad3.set_xlabel("offset(mm)")
ad3.set_ylabel("Difference in Contact_Area(mm2)")
ad3.set_title("Difference Contact Area between PyCo and Formulars")
plt.show()

# Plot of External_load & Max_Pressure

plt.plot(External_load,Max_Pressure,color="green",marker="o",linestyle="dashed",linewidth=2,label="PyCo")
plt.plot(F_External_load,F_Max_Pressure,color="red",marker="s",linewidth=2,label="Formula")
plt.xlabel("External_load (N)")
plt.ylabel("Max_Pressure (MPa)")
plt.title("External_load & Max_Pressure")
plt.legend()
plt.show()

# Plot of External_load & Contact_Area

plt.plot(External_load,Contact_Area,color="green",marker="o",linestyle="dashed",linewidth=2,label="PyCo")
plt.plot(F_External_load,F_Contact_Area,color="red",marker="s",linewidth=2,label="Formula")
plt.xlabel("External_load (N)")
plt.ylabel("Contact_Area (mm2)")
plt.title("External_load & Contact_Area")
plt.legend()
plt.show()